# Create water authority and sectors json files

This file use Kando API to create and save 2 files:
* `water_authority.json` - file containing a dictionary of all points ID by water authorities
* `sectors.json` - file containing a dictionary of all points ID by sectors

**Import Libraries**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import json
import networkx as nx
from kando import kando_client
from matplotlib import pyplot as plt
import pandas as pd
import json
import datetime
%matplotlib inline

**Functions**

In [ ]:
def connect():
    """
    connect to Kando API with json file
    :return: client
    """

    with open('../key.json') as f:
        api_login = json.load(f)

    url = "https://kando-staging.herokuapp.com"
    client = kando_client.client(url, api_login['key'], api_login['secret'])
    return client

**Connect to API**

In [ ]:
client = connect()

**Define water authority numbers**

In [ ]:
water_authority = [
    244, 63, 22, 100, 41, 24, 49, 91, 50, 261, 32, 33, 7, 19, 240, 45, 48, 2,
    38, 242, 3, 190, 36, 58, 243, 83, 9, 5, 60, 72, 34, 8, 4, 195, 17, 252
]

**Get the relevant information of the data points**

In [ ]:
sector_dict = {}
water_authority_dict = {}

In [ ]:
for node in range(6000):
    try:
        start = datetime.datetime(2020, 1, 1, 0, 0).timestamp()
        point = client.get_all(point_id=node, start=start)

        if point['point']['group']['water_authority']['id'] in water_authority:
            print(node, 'Belongs to Israeli water authority')
            print(node, ' has ', len(point['samplings']), ' samplings.')

            if len(point['samplings']) > 0:

                cur_sector = point['point']['group']['sector']['id']
                sector_name = point['point']['group']['sector']['name']
                cur_authority = point['point']['group']['water_authority'][
                    'id']
                authority_name = point['point']['group']['water_authority'][
                    'name']

                print(cur_sector, sector_name, cur_authority, authority_name)

                if (cur_sector, sector_name) not in sector_dict:
                    sector_dict[(cur_sector, sector_name)] = [node]
                else:
                    sector_dict[(cur_sector, sector_name)].append(node)

                if (cur_authority, authority_name) not in water_authority_dict:
                    water_authority_dict[(cur_authority,
                                          authority_name)] = [node]
                else:
                    water_authority_dict[(cur_authority,
                                          authority_name)].append(node)

        else:
            print(node, 'Belongs to foriegn water authority')
    except:
        print(f'no node in {node}')

In [ ]:
sector_dict

In [ ]:
water_authority_dict

In [ ]:
new_dict = {}
for k, v in sector_dict.items():
    new_dict[f'{k[0]}'] = {'name': k[1], 'nodes': v}

new_dict

In [ ]:
with open('sectors.json', 'w') as fp:
    json.dump(new_dict, fp)

In [ ]:
new_dict = {}
for k, v in water_authority_dict.items():
    new_dict[f'{k[0]}'] = {'name': k[1], 'nodes': v}

new_dict

In [ ]:
with open('water_authority.json', 'w') as fp:
    json.dump(new_dict, fp)